<a href="https://colab.research.google.com/github/Rayars/ECE/blob/main/BERT_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [108]:
!pip install sentence-transformers
!pip install sklearn

In [109]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer
import os
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import torch.nn.functional as F   # 激励函数的库
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import random 
import math

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [110]:
#常用函数
def label_translate(target):
  target=list(target)
  for i in range(len(target)):
    if target[i]=='yes':
      target[i]=[1.0,0.0]
    else:
      target[i]=[0.0,1.0]
  target=torch.tensor(target)
  return target

def sentence_embedding(sent):#对整个句子做嵌入，返回子句向量的list
  sentence=[]
  bert=SentenceTransformer('bert-base-nli-mean-tokens')
  for c in sent:
    v=bert.encode(c)
    sentence.append(v)
  return sentence

def positoin_embedding(clause):
  pass

In [147]:
# 定义全局变量
n_epochs = 10     # epoch 的数目
batch_size = 20  # 决定每次读取多少样本

class ECEDataset_sent(Dataset): #每次读取一个句子
    def __init__(self, data_file, transform=None, target_transform=None):
        self.data = pd.read_csv(data_file)
        self.sent_data=self.data.groupby('sent_num')
        self.clauses,self.labels=[],[]
        for i in self.sent_data:
          self.clauses.append(i[1])
          self.labels.append(i[1]['label'])
        self.length=len(self.clauses)
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return self.length

    def __getitem__(self, idx): 
        clause=list(self.clauses[idx]['text'].values)
        clause += ["" for i in range(50-len(clause))]
        label=list(self.labels[idx].values)
        label += ["no" for i in range(50-len(label))]


        if self.transform:
            clause = self.transform(clause)
        if self.target_transform:
            label = self.target_transform(label)
        return clause, label

# 定义训练和数据集，以句子为单位
data=ECEDataset_sent("/content/sample_data/clause_keywords.csv")

#划分训练集和测试集
train_data,test_data=train_test_split(data,train_size=0.8)
print(train_data[0],"\n",test_data[0])


# 创建加载器
train_loader=torch.utils.data.DataLoader(train_data, batch_size = batch_size, num_workers = 2,shuffle=True)
test_loader=torch.utils.data.DataLoader(test_data, batch_size = batch_size, num_workers = 2,shuffle=True)

(['儿子 贪玩 狼爸 拿 竹条 打死 他 案情 永川 的 李某 离婚 后', '带 着 儿子 小鹏 化名 来到 主城 打工', '平时', '李某 一家 靠 制作 销售 豆腐 豆芽菜 维持 生计', '对 儿子 缺少 管护', '贪玩 的 小鹏 让 李某 很 头痛', '1999 年 10 月 15 日晚', '小鹏 放学 后 一直 没 回家', '李某 就 到 儿子 就读 的 学校 附近 寻找', '当晚 8 时许', '李某 找到 儿子', '回家 后', '为 教训 儿子', '李某 把 小鹏 捆绑 在 屋外 院坝 猪圈 横梁 上', '用 竹条 长时间 持续 抽打 儿子 下腹部 背 臀部 等 处', '当晚 10 时许', '小鹏 死亡', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['no', 'no', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no']) 
 (['实际上', '她 这样 做 我 还是 愿意 理解 的', '我 只是 想 不 明白', '为什么 会 这样', '前天', '女方 家长 说 — — 已经 过去 了 半个 多月', '事情 真相 仍然 扑朔迷离', '前天 下午', '为了 弄个 明白', '葵氏 父子 来到 了 郑家', '郑父 说', '出 了 这样 的 事情', '自己 和 妻子 也 对 女儿 感到 十分 失望', '但 女儿 已经 10 多天 没 给 家里 打 过 任何 电话',

In [163]:
# 训练神经网络
def train():
    #定义损失函数和优化器
    lossfunc = nn.CrossEntropyLoss()
    lossfunc.cuda()
    optimizer = torch.optim.SGD(params = model.parameters(), lr = 0.01)
    # 开始训练
    for epoch in range(n_epochs):
        print('Epoch:  {}  \t'.format(epoch+1))
        train_loss = 0.0
        for data,target in tqdm(train_loader):
            tran_data,tran_target=[],[]
            for i in range(batch_size):#经过这个双重循环可以将dataloader取出的数据转置
              sentence=[]
              label=[]
              for d in data:
                sentence.append(d[i])
              for l in target:
                label.append(l[i])
              tran_data.append(sentence)
              tran_target.append(label)
            data,target=tran_data,tran_target
            keys=data
            for i in range(batch_size):
              keys[i]=sentence_embedding(keys[i])
            keys=torch.stack(keys,0)
            querys=keys
            newtarget=[]
            for t in target:
              newtarget.append(label_translate(t))
            target=torch.stack(newtarget,0)
            querys,keys,target=querys.to(device),keys.to(device),target.to(device)
            optimizer.zero_grad()   # 清空上一步的残余更新参数值

            
            output = model(querys,keys,keys)    # 得到预测值 MLP考虑batch_size
            print(output,target)
            print(type(output),type(target))
            print(len(output),len(target))
            print(output[0],target[0])
            for i in range(len(output)):
              loss = lossfunc(output[i].squeeze(0),target[i])  # 计算两者的误差
              loss.backward()         # 误差反向传播, 计算参数更新值
              optimizer.step()        # 将参数更新值施加到 net 的 parameters 上
              train_loss += loss.item()*data.size(0)
        train_loss = train_loss / len(train_loader.dataset)
        print('Training Loss: {:.6f}'.format( train_loss))
        # 每遍历一遍数据集，测试一下准确率
        test()

# 在数据集上测试神经网络
def test():
    TP,FN,FP,TN=0,0,0,0
    P,R,F = 0,0,0
    with torch.no_grad():  # 训练集中不需要反向传播
        for data in tqdm(test_loader):
            keys=data[0]['text'].values
            querys=keys
            querys,keys=sentence_embedding(querys),sentence_embedding(keys)
            target=label_translate(list(data[1].value))
            querys,keys,target=querys.squeeze(0).to(device),keys.squeeze(0).to(device),target.squeeze(0).to(device)
            outputs = model(querys,keys,keys) #outputs: [[0.7,0.3],[0.2,0.8]...]
            _, predicted = torch.max(outputs.data, 1)
            for i in range(len(predicted)):
              if predicted[i] == 1 and target[i] == 1 :
                TP+=1
              elif predicted[i] == 0 and target[i] == 1:
                FN+=1
              elif predicted[i] == 1 and target[i] == 0:
                FP+=1
              elif predicted[i] == 0 and target[i] == 0:
                TN+=1
            P=TP/(TP+FP)
            R=TP/(TP+FN)
            F=2/(1/P+1/R)
    print('Pricision, Recall and F of the network on the test clause: %f %%, %f %%, %f %%' % (
        100.0 * P,100.0*R,100.0*F))
    return 100.0 * P,100.0*R,100.0*F

# 声明感知器网络
model = Attention_MLP(768,1,1)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#print(device)
model.to(device)
train()

Epoch:  1  	


  0%|          | 0/85 [01:44<?, ?it/s]

torch.Size([20, 50, 768])
[tensor([0.4868, 0.5132], grad_fn=<SqueezeBackward1>), tensor([0.4873, 0.5127], grad_fn=<SqueezeBackward1>), tensor([0.4870, 0.5130], grad_fn=<SqueezeBackward1>), tensor([0.4866, 0.5134], grad_fn=<SqueezeBackward1>), tensor([0.4873, 0.5127], grad_fn=<SqueezeBackward1>), tensor([0.4867, 0.5133], grad_fn=<SqueezeBackward1>), tensor([0.4871, 0.5129], grad_fn=<SqueezeBackward1>), tensor([0.4868, 0.5132], grad_fn=<SqueezeBackward1>), tensor([0.4863, 0.5137], grad_fn=<SqueezeBackward1>), tensor([0.4853, 0.5147], grad_fn=<SqueezeBackward1>), tensor([0.4859, 0.5141], grad_fn=<SqueezeBackward1>), tensor([0.4876, 0.5124], grad_fn=<SqueezeBackward1>), tensor([0.4870, 0.5130], grad_fn=<SqueezeBackward1>), tensor([0.4871, 0.5129], grad_fn=<SqueezeBackward1>), tensor([0.4863, 0.5137], grad_fn=<SqueezeBackward1>), tensor([0.4876, 0.5124], grad_fn=<SqueezeBackward1>), tensor([0.4877, 0.5123], grad_fn=<SqueezeBackward1>), tensor([0.4891, 0.5109], grad_fn=<SqueezeBackward1>), t

RuntimeError: ignored

In [112]:
# 感知机网络
class MLP(nn.Module):   
    def __init__(self):
        super(MLP,self).__init__()   
        
        self.fc1 = torch.nn.Linear(768,256)  
        self.fc2 = torch.nn.Linear(256,128)  
        self.fc3 = torch.nn.Linear(128,64)   
        self.fc4 = torch.nn.Linear(64,32)   
        self.fc5 = torch.nn.Linear(32,8)
        self.fc6 = torch.nn.Linear(8,2)
        

        
    def forward(self,din):
        # 前向传播， 输入值：din, 返回值 dout
        dout = F.relu(self.fc1(din))   # 使用 relu 激活函数
        dout = F.relu(self.fc2(dout))
        dout = F.relu(self.fc3(dout))
        dout = F.relu(self.fc4(dout))
        dout = F.relu(self.fc5(dout))
        dout = F.softmax(self.fc6(dout), dim=1)  # 输出层使用 softmax 激活函数
        # 2个数字实际上是2个类别，输出是概率分布，最后选取概率最大的作为预测值输出
        return dout

In [113]:
class dot_attention(nn.Module):
    """ 点积注意力机制"""

    def __init__(self, attention_dropout=0.0):
        super(dot_attention, self).__init__()
        self.dropout = nn.Dropout(attention_dropout)
        self.softmax = nn.Softmax(dim=2)

    def forward(self, q, k, v, scale=None, attn_mask=None):
        """
        前向传播
        :param q:
        :param k:
        :param v:
        :param scale:
        :param attn_mask:
        :return: 上下文张量和attention张量。
        """
        attention = torch.bmm(q, k.transpose(1, 2))
        if scale:
            attention = attention * scale        # 是否设置缩放
        if attn_mask:
            attention = attention.masked_fill(attn_mask, -np.inf)     # 给需要mask的地方设置一个负无穷。
        # 计算softmax
        attention = self.softmax(attention)
        # 添加dropout
        attention = self.dropout(attention)
        # 和v做点积。
        context = torch.bmm(attention, v)
        return context, attention

In [126]:
#多头自注意力机制 
class MultiHeadAttention(nn.Module):
    """ 多头自注意力"""
    def __init__(self, model_dim=768, num_heads=2, dropout=0.0):
        super(MultiHeadAttention, self).__init__()

        self.dim_per_head = model_dim//num_heads   # 每个头的维度
        self.num_heads = num_heads
        self.linear_k = nn.Linear(model_dim, self.dim_per_head * num_heads)
        self.linear_v = nn.Linear(model_dim, self.dim_per_head * num_heads)
        self.linear_q = nn.Linear(model_dim, self.dim_per_head * num_heads)

        self.dot_product_attention = dot_attention(dropout)

        self.linear_final = nn.Linear(model_dim, model_dim)
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(model_dim)         # LayerNorm 归一化。

    def forward(self, key, value, query, attn_mask=None):
        # 残差连接
        residual = query

        dim_per_head = self.dim_per_head
        num_heads = self.num_heads
        batch_size = key.size(0)

        # 线性映射。
        key = self.linear_k(key)
        value = self.linear_v(value)
        query = self.linear_q(query)

        # 按照头进行分割
        key = key.view(batch_size * num_heads, -1, dim_per_head)
        value = value.view(batch_size * num_heads, -1, dim_per_head)
        query = query.view(batch_size * num_heads, -1, dim_per_head)

        if attn_mask:
            attn_mask = attn_mask.repeat(num_heads, 1, 1)

        # 缩放点击注意力机制
        scale = (key.size(-1) // num_heads) ** -0.5
        context, attention = self.dot_product_attention(query, key, value, scale, attn_mask)

        # 进行头合并 concat heads
        context = context.view(batch_size, -1, dim_per_head * num_heads)

        # 进行线性映射
        output = self.linear_final(context)

        # dropout
        output = self.dropout(output)

        # 添加残差层和正则化层。
        output = self.layer_norm(residual + output)

        return output, attention
#BERT_MultiHeadSelfAttention模型
class Attention_MLP(nn.Module):

    def __init__(self,model_dim=768, num_heads=1, dropout=0.0):
        super().__init__()
        self.attention=MultiHeadAttention(model_dim=768, num_heads=1, dropout=0.0)
        self.MLP=MLP()

    def forward(self,query,key,value):
        output,attention=self.attention.forward(key,query,value)
        print(output.shape)
        result=[]
        for clause in output[0].squeeze(0):
          result.append(self.MLP(clause.unsqueeze(0)).squeeze(0))     
        return result